<a href="https://colab.research.google.com/github/neuralsrg/SequenceModels/blob/main/manual_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manual implementation of RNN in numpy

In [9]:
import numpy as np 
import scipy.special

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

## RNN

### RNN unit
![image](https://i.imgur.com/7PAU2pP.png)

In [ ]:
def rnn_unit(x, a_prev, parameters):
  '''
  Implements signle RNN unit forward pass

  Args:
  x -- input vector
  a_prev -- activation from previous RNN unit
  parameters -- python dictionary with keys 'Waa', 'Wax', 'ba', 'Wya', 'by'

  Returns:
  y_pred -- softmax activation outputs
  a -- current RNN unit hidden state
  cache -- tuple containing (a, a_prev, x, parameters)
  '''
  Waa, Wax, ba, Wya, by = parameters['Waa'], parameters['Wax'], parameters['ba'], \
                          parameters['Wya'], parameters['by']

  a = Waa @ a_prev + Wax @ x + ba
  a = np.tanh(a)

  y_pred = scipy.special.softmax(Wya @ a + by, axis=0)

  cache = (a, a_prev, x, parameters)

  return y_pred, a, cache

### RNN forward pass

In [ ]:
def rnn(x, a_init, parameters):
  '''
  Implements RNN forward pass

  Args:
  x -- tensor of shape (n_x, m, T_x), where n_x - num of features, m - batch size,
      T_x - num of time steps
  a_init -- initial rnn hidden state
  parameters -- python dictionary with keys 'Waa', 'Wax', 'ba', 'Wya', 'by'

  Returns:
  Y_pred -- tensor of shape (n_y, m, T_x)
  A -- tensor of hidden states of shape (n_a, m, T_x)
  caches -- (list_of_caches, x)
  '''
  n_x, m, T_x = x.shape
  a = a_init
  n_y, n_a = parameters['Wya'].shape

  Y_pred = np.empty((n_y, m, 0))
  A = np.empty((n_a, m, 0))
  caches = []

  for time_step in range(T_x):
    y_pred, a, cache = rnn_unit(x[:, :, time_step], a, parameters)

    Y_pred = np.append(Y_pred, y_pred[..., np.newaxis], axis=-1)
    A = np.append(A, a[..., np.newaxis], axis=-1)
    caches.append(cache)

  return Y_pred, A, (caches, x)

### RNN backpropagation

![image](https://i.imgur.com/x2jxcSq.png)

$$
\begin{align}
\displaystyle a^{\langle t \rangle} &= \tanh(W_{ax} x^{\langle t \rangle} + W_{aa} a^{\langle t-1 \rangle} + b_{a})\tag{-} \\[8pt]
\displaystyle \frac{\partial \tanh(x)} {\partial x} &= 1 - \tanh^2(x) \tag{-} \\[8pt]
\displaystyle {dtanh} &= da_{next} * ( 1 - \tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a})) \tag{0} \\[8pt]
\displaystyle  {dW_{ax}} &= dtanh \cdot x^{\langle t \rangle T}\tag{1} \\[8pt]
\displaystyle dW_{aa} &= dtanh \cdot a^{\langle t-1 \rangle T}\tag{2} \\[8pt]
\displaystyle db_a& = \sum_{batch}dtanh\tag{3} \\[8pt]
\displaystyle dx^{\langle t \rangle} &= { W_{ax}}^T \cdot dtanh\tag{4} \\[8pt]
\displaystyle da_{prev} &= { W_{aa}}^T \cdot dtanh\tag{5}
\end{align}
$$

In [ ]:
def rnn_unit_backward(da, cache):
  '''
  Implements the backward pass for a single rnn unit based on it's cache

  Args:
  da -- dJ/da (da_next from the picture above)
  cache -- cache from rnn forward pass

  Returns:
  grads -- python dictionary with keys: 'dba', 'dWax', 'dx' (for deep RNNs),
      'dWaa', 'dWa_prev', 'da_prev'
  '''
  a, a_prev, x, parameters = cache
  Waa, Wax, ba, Wya, by = parameters['Waa'], parameters['Wax'], parameters['ba'], \
                          parameters['Wya'], parameters['by']

  grads = {}

  dz = da * (1 - np.square(a))

  grads['dba'] = np.sum(dz, axis=-1, keepdims=True)
  grads['dWaa'] = dz @ a_prev.T # it is basically a sum over mini-batch sample gradients
  grads['da_prev'] = Waa.T @ dz # shape (n_a, m)

  grads['dWax'] = dz @ x.T
  grads['dx'] = Wax.T @ dz

  return grads

In [ ]:
def rnn_backward(da, caches):
  '''
  Implements the backward pass for the entire RNN

  Args:
  da -- dJ/da (from softmax) of shape (n_a, m, T_x) computed elsewhere
  caches -- output from rnn()

  Returns:
  grads -- python dictionary with keys 
    'dx' : shape(n_x, m, T_x)
    'da_init' : shape(n_a, m)
    'dWax' : shape(n_a, n_x)
    'dWaa' : shape(n_a, n_a)
    'dba' : shape(n_a, 1)
  '''
  caches, x = caches
  a, a_init, x, parameters = caches[0]

  n_a, m, T_x = da.shape
  n_x, _ = x.shape

  dx = np.empty((n_x, m, 0))
  dWax = np.zeros((n_a, n_x))
  dWaa = np.zeros((n_a, n_a))
  dba = np.zeros((n_a, 1))

  # to save the last value
  da_prev = np.zeros((n_a, m))

  for time_step in range(T_x - 1, -1, -1):
    grads = rnn_unit_backward(da[:, :, time_step] + da_prev, caches[time_step])
    dba_t, dWaa_t, da_prev, dWax_t, dx_t = grads['dba'], grads['dWaa'], \
        grads['da_prev'], grads['dWax'], grads['dx']
    
    dx = np.append(dx_t[..., np.newaxis], dx, axis=-1)
    dWax += dWax_t
    dWaa += dWaa_t
    dba += dba_t
  
  grads = {"dx": dx, "da0": da_prev, "dWax": dWax, "dWaa": dWaa, "dba": dba}

  return grads

## LSTM (Long Short-Term Memory) Network

### LSTM unit

$\tilde c^{<t>} = tanh(W_c[a^{<t-1>}, x^{<t>}] + b_c)$

$\Gamma_u=\sigma(W_u[a^{<t-1>}, x^{<t>}] + b_u)\text{   } (update)$

$\Gamma_f=\sigma(W_f[a^{<t-1>}, x^{<t>}] + b_f) \text{   } (forget)$

$\Gamma_o=\sigma(W_o[a^{<t-1>}, x^{<t>}] + b_o) \text{   } (output)$

$c^{<t>} = \Gamma_u*\tilde c^{<t>} + \Gamma f *c^{<t-1>}$

$a^{<t>} = \Gamma _o *tanh(c^{<t>})$


![image](https://i.imgur.com/bT5bsFa.png)



In [48]:
def lstm_unit(x, a_prev, c_prev, parameters):
  '''
  Implements a single lstm time step

  Args:
  x -- input data of shape (n_x, m)
  a_prev -- previous hidden state of shape (n_a, m)
  c_prev -- previous memory state of shape (n_a, m)
  parameters -- python dictionary with keys: 'Wc', 'bc', 'Wu', 'bu', 'Wf', 'bf',
       'Wo', 'bo', 'Wy', 'by'

  Returns:
  y_pred -- predictions of shape (n_y, m)
  a -- current hidden state of shape (n_a, m)
  c -- current memory state of shape (n_a, m)
  cache -- (a, c, a_prev, c_prev, forget_gate, update_gate, candidate,
      output_gate, x, parameters) - for backprop
  '''
  n_x, m = x.shape

  Wc, bc, Wu, bu, Wf, bf, Wo, bo, Wy, by = parameters['Wc'], parameters['bc'], \
      parameters['Wu'], parameters['bu'], parameters['Wf'], parameters['bf'], \
      parameters['Wo'], parameters['bo'], parameters['Wy'], parameters['by']

  n_y, n_a = Wy.shape
  concatenated = np.concatenate((a_prev, x), axis=0)

  candidate = np.tanh(Wc @ concatenated + bc)

  update_gate = sigmoid(Wu @ concatenated + bu)
  forget_gate = sigmoid(Wf @ concatenated + bf)
  output_gate = sigmoid(Wo @ concatenated + bo)

  c = update_gate * candidate + forget_gate * c_prev
  a = output_gate * np.tanh(c)

  y_pred = scipy.special.softmax(Wy @ a + by, axis=0)
  cache = (a, c, a_prev, c_prev, forget_gate, update_gate, candidate, \
      output_gate, x, parameters)
  
  return y_pred, a, c, cache

### LSTM forward pass

![image](https://i.imgur.com/SJQeZgS.png)

In [25]:
def lstm(X, a_init, parameters):
  '''
  Implements lstm forward propagation

  Args:
  X -- input data of shape (n_x, m, T_x)
  a_init -- initial hidden state
  parameters -- python dictionary, same as above

  Returns:
  Y_pred -- predictions of shape (n_y, m, T_x)
  A -- hidden states of shape (n_a, m, T_x)
  C -- memory cells of shape (n_a, m, T_x)
  caches -- ([caches], X)
  '''
  n_x, m, T_x = X.shape
  n_y, n_a = parameters['Wy'].shape

  Y_pred = np.empty((n_y, m, 0))
  A = np.empty((n_a, m, 0))
  C = np.empty((n_a, m, 0))
  caches = []

  a = a_init
  c = np.zeros((n_a, m))

  for time_step in range(T_x):

    y_pred, a, c, cache = lstm_unit(X[:, :, time_step], a, c, parameters)
    Y_pred = np.concatenate((Y_pred, y_pred[..., np.newaxis]), axis=-1)
    A = np.concatenate((A, a[..., np.newaxis]), axis=-1)
    C = np.concatenate((C, c[..., np.newaxis]), axis=-1)
    caches.append(cache)

  return Y_pred, A, C, (caches, X)

### LSTM backpropagation

![image](https://i.imgur.com/GWuqEdC.png)


\begin{align}
d\gamma_o^{\langle t \rangle} &= da_{next}*\tanh(c_{next}) * \Gamma_o^{\langle t \rangle}*\left(1-\Gamma_o^{\langle t \rangle}\right)\tag{7} \\[8pt]
dp\widetilde{c}^{\langle t \rangle} &= \left(dc_{next}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c_{next})) * \Gamma_u^{\langle t \rangle} * da_{next} \right) * \left(1-\left(\widetilde c^{\langle t \rangle}\right)^2\right) \tag{8} \\[8pt]
d\gamma_u^{\langle t \rangle} &= \left(dc_{next}*\widetilde{c}^{\langle t \rangle} + \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c_{next})) * \widetilde{c}^{\langle t \rangle} * da_{next}\right)*\Gamma_u^{\langle t \rangle}*\left(1-\Gamma_u^{\langle t \rangle}\right)\tag{9} \\[8pt]
d\gamma_f^{\langle t \rangle} &= \left(dc_{next}* c_{prev} + \Gamma_o^{\langle t \rangle} * (1-\tanh^2(c_{next})) * c_{prev} * da_{next}\right)*\Gamma_f^{\langle t \rangle}*\left(1-\Gamma_f^{\langle t \rangle}\right)\tag{10}
\end{align}


$dW_f = d\gamma_f^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{11}$

$dW_u = d\gamma_u^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{12}$

$dW_c = dp\widetilde c^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{13}$

$dW_o = d\gamma_o^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{14}$

$\displaystyle db_f = \sum_{batch}d\gamma_f^{\langle t \rangle}\tag{15}$

$\displaystyle db_u = \sum_{batch}d\gamma_u^{\langle t \rangle}\tag{16}$

$\displaystyle db_c = \sum_{batch}dp\tilde c^{\langle t \rangle}\tag{17}$

$\displaystyle db_o = \sum_{batch}d\gamma_o^{\langle t \rangle}\tag{18}$

$da_{prev} = W_f^T d\gamma_f^{\langle t \rangle} + W_u^T   d\gamma_u^{\langle t \rangle}+ W_c^T dp\widetilde c^{\langle t \rangle} + W_o^T d\gamma_o^{\langle t \rangle} \tag{19}$

$dc_{prev} = dc_{next}*\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh^2(c_{next}))*\Gamma_f^{\langle t \rangle}*da_{next} \tag{20}$

$dx^{\langle t \rangle} = W_f^T d\gamma_f^{\langle t \rangle} + W_u^T  d\gamma_u^{\langle t \rangle}+ W_c^T dp\widetilde c^{\langle t \rangle} + W_o^T d\gamma_o^{\langle t \rangle}\tag{21}$




In [59]:
def lstm_unit_backward(da, dc, cache):
  '''
  Implements backward pass for a signle LSTM unit

  Args:
  da -- dJ/da_next
  dc -- dJ/dc_next
  cache -- output from LSTM unit forward prop

  Returns:
  grads with keys: 'dx', 'da_prev', 'dc_prev', 'dWf', 'dbf', 'dWu', 'dbu', 
      'dWc', 'dbc', 'dWo', 'dbo'
  '''
  a, c, a_prev, c_prev, forget_gate, update_gate, candidate, \
      output_gate, x, parameters = cache

  n_x, m = x.shape
  n_a = parameters['Wc'].shape[0]

  d_output_gate = da * np.tanh(c) * output_gate * (1 - output_gate)
  d_candidate = (dc * update_gate + output_gate * (1 - np.tanh(c) ** 2) * \
      update_gate * da) * (1 - candidate ** 2)
  d_update_gate = (dc * candidate + output_gate * (1 - np.tanh(c) ** 2) * \
      candidate * da) * update_gate * (1 - update_gate)
  d_forget_gate = (dc * c_prev + output_gate * (1 - np.tanh(c) ** 2) * \
      c_prev * da) * forget_gate * (1 - forget_gate)

  concatenated = np.concatenate((a_prev, x), axis=0)

  dWf = d_forget_gate @ concatenated.T
  dWu = d_update_gate @ concatenated.T
  dWc = d_candidate @ concatenated.T
  dWo = d_output_gate @ concatenated.T

  dbf = np.sum(d_forget_gate, axis=1, keepdims=True)
  dbu = np.sum(d_update_gate, axis=1, keepdims=True)
  dbc = np.sum(d_candidate, axis=1, keepdims=True)
  dbo = np.sum(d_output_gate, axis=1, keepdims=True)

  da_prev = parameters['Wf'][:, :n_a].T @ d_forget_gate + parameters['Wu'][:, :n_a].T @ d_update_gate + \
      parameters['Wc'][:, :n_a].T @ d_candidate + parameters['Wo'][:, :n_a].T @ d_output_gate

  dc_prev = dc * forget_gate + output_gate * (1 - np.tanh(c) ** 2) * \
      forget_gate * da
  dx = parameters['Wf'][:, n_a :].T @ d_forget_gate + parameters['Wu'][:, n_a :].T @ d_update_gate + \
      parameters['Wc'][:, n_a :].T @ d_candidate + parameters['Wo'][:, n_a :].T @ d_output_gate

  grads = {"dxt": dx, "da_prev": da_prev, "dc_prev": dc_prev, "dWf": dWf,"dbf": dbf, "dWu": dWu,"dbu": dbu,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

  return grads

In [60]:
def lstm_backward(da, caches):
  '''
  Implements LSTM backward pass

  Args:
  da -- dJ/da from Dense -> softmax
  caches -- output from lstm()

  Returns:
  grads with keys: 'dx', 'da_prev', 'dc_prev', 'dWf', 'dbf', 'dWu', 'dbu', 
      'dWc', 'dbc', 'dWo', 'dbo'
  '''
  caches, X = caches
  a, c, a_prev, c_prev, forget_gate, update_gate, candidate, \
      output_gate, x, parameters = caches[0]

  n_x, m, T_x = X.shape
  n_a, m, T_x = da.shape
  #n_a = parameters['Wc'].shape[0]

  dx = np.empty((n_x, m, 0))
  a_init = np.zeros((n_a, m))

  dWf = np.zeros((n_a, n_a + n_x))
  dWu = np.zeros((n_a, n_a + n_x))
  dWc = np.zeros((n_a, n_a + n_x))
  dWo = np.zeros((n_a, n_a + n_x))

  dbf = np.zeros((n_a, 1))
  dbu = np.zeros((n_a, 1))
  dbc = np.zeros((n_a, 1))
  dbo = np.zeros((n_a, 1))

  da_next = np.zeros_like(da[:, :, 0])
  dc = np.zeros_like(da[:, :, 0])

  for time_step in range(T_x - 1, -1, -1):
    grads = lstm_unit_backward(da[:, :, time_step] + da_next, dc, caches[time_step])
    
    da_next = grads['da_prev']
    dc = grads['dc_prev']
    dx = np.concatenate((grads['dxt'][..., np.newaxis], dx), axis=-1)

    dWf += grads['dWf']
    dWu += grads['dWu']
    dWc += grads['dWc']
    dWo += grads['dWo']

    dbf += grads['dbf']
    dbu += grads['dbu']
    dbc += grads['dbc']
    dbo += grads['dbo']

  grads = {"dx": dx, "da_init": da_next, "dWf": dWf,"dbf": dbf, "dWu": dWu,
               "dbu": dbu, "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

  return grads 